In [1]:
import pandas as pd
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split

In [2]:
import lightgbm as lgb

# LightGBM model and parameters

In [7]:
params = {}
params['learning_rate'] = 0.09043007325486686
#params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 45
params['min_data'] = 50
params['max_depth'] = 10
#params['device'] = 'gpu'

# Hyper parameter tuning from kaggle kernel:
{'boosting_type': 0,
 'colsample_by_tree': 0.7073741758938382,
 'gdbt_subsample': 0.576055003450302,
 'is_unbalance': 0,
 'learning_rate': 0.09043007325486686,
 'min_child_samples': 270.0,
 'num_leaves': 45.0,
 'reg_alpha': 0.14138716535199491,
 'reg_lambda': 0.2889595293526205,
 'subsample_for_bin': 200000.0}



In [3]:
# Prepare datasets
X = pd.read_csv('result_data/train_app_prev_b_pos_credit_merged_corr.csv')
Y  = X['TARGET']
X  = X.drop(columns=['TARGET'],axis=1)

In [ ]:
X.shape

In [ ]:
test_data.shape

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
d_train = lgb.Dataset(X_train, label=y_train)

In [8]:
clf = lgb.train(params, d_train, 200)

In [9]:
y_pred=clf.predict(X_test)

In [10]:
type(y_test.values)
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [11]:
auc(fpr, tpr)

0.7809199400958419

# Results

1. app_precessed data  :   74.23      test_roc: 70
2. train_app_bureau_merged: 0.7298677  test_roc: 0.721
3. train_app_bu_mer_corr_drp: 0.73263626999423  test_roc: 0.724
4. Same: Leaves 60 : 0.7371016183398422
5. Same: + LR : 0.09043007325486686 + leaves 45: 0.7409575603828592
6. Same + 270 estimators : 0.7591253403483376    test_roc: 0.742
7. Same + 230 estimators: 0.7601499389379056     test_roc:
8. Same + 200 : 0.7605678355570721
9. Domain Features + same + no_crr_drp : 0.7736541538631981
10. Domain Features + same + crr_drp : 0.76
11. merged app , bureau , prev app , pos : 0.86369184
12. merged app, bureau , prev app, pos, installment : 0.7825008494716269

# Predict test data


In [13]:
test_data = pd.read_csv('result_data/test_app_prev_b_pos_credit_merged_corr.csv')

In [14]:
predict_data = clf.predict(test_data)

In [15]:
df_pred = pd.DataFrame(predict_data)
df_pred['SK_ID_CURR'] =test_data['SK_ID_CURR']
df_pred.head()

KeyError: 'SK_ID_CURR'

In [ ]:
df_pred.set_index('SK_ID_CURR',inplace=True)
df_pred.rename(columns={0:'TARGET'},inplace=True)

In [25]:
df_pred.head()

,TARGET
SK_ID_CURR,
100001,0.017916
100005,0.094166
100013,0.014952
100028,0.039355
100038,0.101310


In [26]:
df_pred.to_csv('test output/pred_bureu_prev_pos_corr_lbm_fifth__hyper_200_est.csv',index=True)

# XGBoost Model

In [ ]:
from xgboost import XGBClassifier
import numpy as np

In [ ]:
# Prepare datasets
X = pd.read_csv('result_data/train_app_bureau_merged.csv')
test_data = pd.read_csv('result_data/test_app_bureau_merged.csv')
Y  = X['TARGET']
X  = X.drop(columns=['TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Create a xgboost model
model = XGBClassifier(learning_rate=0.09043007325486686,n_jobs=-1,n_estimators=200)
model.fit(X_train,y_train)

In [ ]:
y_predict = model.predict_proba(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [ ]:
auc(fpr, tpr)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_roc(y_test.values, y_predict)
#plt.show()